In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [111]:
movie_gross = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')
name_basics = pd.read_csv('./zippedData/imdb.name.basics.csv.gz')
title_akas = pd.read_csv('./zippedData/imdb.title.akas.csv.gz')
title_basics = pd.read_csv('./zippedData/imdb.title.basics.csv.gz')
title_crew = pd.read_csv('./zippedData/imdb.title.crew.csv.gz')
title_principals = pd.read_csv('./zippedData/imdb.title.principals.csv.gz')
title_ratings = pd.read_csv('./zippedData/imdb.title.ratings.csv.gz')
movie_info= pd.read_csv('./zippedData/rt.movie_info.tsv.gz', sep='\t')
rt_reviews= pd.read_csv('./zippedData/rt.reviews.tsv.gz', sep='\t', encoding='windows-1254')
budgets = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')

In [112]:
# print(movie_gross.keys())
# print(name_basics.keys())
# print(title_basics.keys())
# print(title_akas.keys())
# print(title_crew.keys())
# print(title_principals.keys())
# print(title_ratings.keys())
# print(movie_info.keys())
# print(budgets.keys())


In [8]:
# print(movie_gross.isna().sum())
# print(name_basics.isna().sum())
# print(title_basics.isna().sum())
# print(title_akas.isna().sum())
# print(title_crew.isna().sum())
# print(title_principals.isna().sum())
# print(title_ratings.isna().sum())
# print(movie_info.isna().sum())
# print(budgets.isna().sum())

In [113]:
#converting all values in budgets sheets to ints by removing commas and $
budgets['domestic_gross'] = budgets['domestic_gross'].str.replace('$' , '')
budgets['domestic_gross'] = budgets['domestic_gross'].str.replace(',', '')
budgets['domestic_gross'] = budgets['domestic_gross'].astype(int)

budgets['production_budget'] = budgets['production_budget'].str.replace('$' , '')
budgets['production_budget'] = budgets['production_budget'].str.replace(',', '')
budgets['production_budget'] = budgets['production_budget'].astype(int)

budgets['worldwide_gross'] = budgets['worldwide_gross'].str.replace('$' , '')
budgets['worldwide_gross'] = budgets['worldwide_gross'].str.replace(',', '')
budgets['worldwide_gross'] = budgets['worldwide_gross'].astype(int)


In [114]:
#creating new lines showing the difference between gross and cost, aka net, as well as a profit percentage calculation showing net over budget
budgets['domestic_net']= budgets['domestic_gross']- budgets['production_budget']
budgets['worldwide_net']= budgets['worldwide_gross']- budgets['production_budget']
budgets['profit_percentage'] = (budgets['worldwide_net']/budgets['production_budget'])*100

In [115]:
#combining ratings and basics to produce a dataframe that matches titles to ratings
titlerate = title_ratings.merge(title_basics, on= 'tconst', how= 'outer')

In [116]:
#dropping null values from the combined table
titlerate.dropna(subset= ['averagerating'], inplace = True)

In [117]:
titlerate.isna().sum()

tconst                0
averagerating         0
numvotes              0
primary_title         0
original_title        0
start_year            0
runtime_minutes    7620
genres              804
dtype: int64

In [118]:
titlerate.shape

(73856, 8)

In [119]:
#merging the two rotten tomatoes dataframes
tomatoes = rt_reviews.merge(movie_info, on='id', how='outer')

In [120]:
tomatoes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54857 entries, 0 to 54856
Data columns (total 19 columns):
id              54857 non-null int64
review          48869 non-null object
rating_x        40915 non-null object
fresh           54432 non-null object
critic          51710 non-null object
top_critic      54432 non-null float64
publisher       54123 non-null object
date            54432 non-null object
synopsis        54680 non-null object
rating_y        54769 non-null object
genre           54764 non-null object
director        49339 non-null object
writer          45426 non-null object
theater_date    53411 non-null object
dvd_date        53411 non-null object
currency        33351 non-null object
box_office      33351 non-null object
runtime         54001 non-null object
studio          40204 non-null object
dtypes: float64(1), int64(1), object(17)
memory usage: 8.4+ MB


In [121]:
#changing the theater dates to datetime format so we can measure revenue against release date
tomatoes['theater_date'] = pd.to_datetime(tomatoes['theater_date'])


In [103]:
tomatoes.head()

,id,review,rating_x,fresh,critic,top_critic,publisher,synopsis,rating_y,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0.0,Patrick Nabarro,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,"Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0.0,io9.com,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,"Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0.0,Stream on Demand,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,"Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0.0,MUBI,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,"Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0.0,Cinema Scope,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,"Jan 1, 2013",$,"600,000",108 minutes,Entertainment One


In [122]:
#dropping the date column to avoid confusion- it's the date of the review, whereas theater_date is the release
tomatoes.drop(['date'], axis = 1, inplace = True)
#reformatting the box office numbers to ints
tomatoes['box_office'] = tomatoes['box_office'].str.replace(',','')
#dropping null values here- it substantially cuts into our dataset and may not be the right choice, but I don't know how to switch to ints and leave null values. This column seems particularly valuable to map. Discuss?
tomatoes.dropna(subset= ['box_office'], inplace= True)
tomatoes['box_office'] = tomatoes['box_office'].astype(int)

In [123]:
tomatoes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33351 entries, 0 to 54853
Data columns (total 18 columns):
id              33351 non-null int64
review          32157 non-null object
rating_x        25538 non-null object
fresh           33310 non-null object
critic          32085 non-null object
top_critic      33310 non-null float64
publisher       33104 non-null object
synopsis        33351 non-null object
rating_y        33351 non-null object
genre           33351 non-null object
director        29405 non-null object
writer          27666 non-null object
theater_date    33237 non-null datetime64[ns]
dvd_date        33237 non-null object
currency        33351 non-null object
box_office      33351 non-null int64
runtime         33169 non-null object
studio          31005 non-null object
dtypes: datetime64[ns](1), float64(1), int64(2), object(14)
memory usage: 4.8+ MB


In [125]:
#since all currency seems to be in dollars we don't need to bother with a conversion here
tomatoes.currency.unique()

array(['$'], dtype=object)